<a href="https://colab.research.google.com/github/SamHollings/Warhammer_chat/blob/main/Warhammer_40k_abominable_intelligence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

See this guide: https://colab.research.google.com/drive/181Kummxd8yOyRqFu8I0aqjs2aqnOy4Fu?usp=sharing#scrollTo=ozGyfQHXepiJ

See this other guide@ https://github.com/devsentient/examples/blob/main/LLMs/confluence_app/confluenceQA.ipynb

And another very helpful guide: https://www.shakudo.io/blog/building-confluence-kb-qanda-app-langchain-chatgpt

# Setup

First need to install the modules - the version of numpy in google collab conflicts with the other modules, and the best way to resolve this was the uninstall first.

In [ ]:
%pip uninstall numpy -y

Found existing installation: numpy 1.23.5
Uninstalling numpy-1.23.5:
  Successfully uninstalled numpy-1.23.5


In [ ]:
%pip install chromadb openai anthropic langchain numpy==1.23 datasets python-dotenv sentence-transformers --no-cache-dir -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.8/437.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 139.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.3/796.3 kB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 184.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 173.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 200.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 164.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 187.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 157.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 96.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 163.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 175.3 MB/s eta

This section gets your API keys - it will ask to upload a ".env" file which contains API keys as follow:

```
name_of_key=value
name_of_second_key=value
```





In this code we use an "openai_key" and an "anthropic_key" - the openai key isn't actually use anymore.

In [ ]:
from google.colab import files
import os

# Check if file already exists
if os.path.exists('.env'):
    os.remove('.env')

# Upload file
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

# Rename file
try:
    os.rename(file_name, '.env')
    print('File uploaded and renamed successfully.')
except:
    print('Error renaming file.')

Saving .env to .env
File uploaded and renamed successfully.


In [ ]:
from dotenv import load_dotenv
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI

# Load variables from .env file
load_dotenv('.env')

# Use variables
#os.environ["OPENAI_API_KEY"] = os.getenv('openai_key')
os.environ["ANTHROPIC_API_KEY"] = os.getenv('anthropic_key')

# Database

## Load existing vectorstore

In [ ]:
# ToDo: Add some functions which can simply load the vector store
# The below will try and load from the persist_directory, if it exists.
from langchain.vectorstores import Chroma
from langchain.schema.document import Document
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-mpnet-base-v2") #embedding_functions.DefaultEmbeddingFunction()

persist_directory = 'db'
vectorstore = Chroma(persist_directory=persist_directory, embedding_function=embedding)

## Format raw text for vector store

I pulled this XML from lexicanum:
- https://wh40k.lexicanum.com/wiki/Special:Export

"imperium" is a good category with ~200 interesting pages, so good place to start.

Going forward, I need to think of a better way of getting more pages.

In [ ]:
# load xml file
import xml.etree.ElementTree as ET

In [ ]:
def clean_document(document_string):
  """The raw XML (especially for mediawiki) has a lot of characters were can remove"""
  return (document_string
          .replace("{{","")
          .replace("}}","")
          .replace("\'","")
          .replace("[[","")
          .replace("]]","")
          .replace("***","")
  )

In [ ]:
def extract_lexicanum_xml_pages(xml_file):
  """split the xml_file by page and return a list of pages (or save each page as a separate file?)"""
  ns = {'site': 'http://www.mediawiki.org/xml/export-0.10/',}
  tree = ET.parse(xml_file)
  root = tree.getroot()

  pages = root.findall('site:page',ns)

  return [process_page(page,ns) for page in pages]

def process_page(xml_page: ET.Element,ns):
  """Take xml page, chunk it as needed for LLM vector store, add appropriate metadata, and return in a simple text format
  e.g. JSON: page_content:..., metadata: title: ...., paragraph/section:..."""
  page_title = xml_page.find('site:title',ns).text
  page_content = xml_page.find('site:revision',ns).find('site:text',ns).text

  return dict(page_content=clean_document(page_content), metadata={"title":page_title})

In [ ]:
documents = extract_lexicanum_xml_pages("/content/Warhammer+40k+-+Lexicanum-20230929124329.xml")

In [ ]:
len(documents)

251

## split documents into chunks

We need to split the documents, as they are too long on their own to be useful to the LLM

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

#  convert the documents to chroma document format
chroma_documents =  [Document(**document) for document in documents]

# split them into more manageable chunks (some of the above documents are quite long)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)

texts = text_splitter.split_documents(chroma_documents)

In [ ]:
print(texts[0])
print(texts[4])

page_content='ImperiumPortal\notheruses|USE=Human Empire|OTHERUSE=Magazine|OTHERPAGE=Warhammer 40,000: Imperium Magazine\n{|hspace=5 VSPACE=5 align=right cellpadding=3 cellspacing=3 style="clear:right; margin:5px; border:1px solid #090909"\n|colspan=2 align="center"|Imperium of Man\n|-\n|align=center colspan=2 style="font-size:85%;font:85%"|Image:Imperial Eagle.jpg|250px\n|-\n|cellpadding=2 cellspacing=1 style="font-size:85%;font:85%"|Capital:\n|cellpadding=2 cellspacing=1 style="font-size:85%;font:85%"|Terra\n|-\n|cellpadding=2 cellspacing=1 style="font-size:85%;font:85%"|Official languages:\n|cellpadding=2 cellspacing=1 style="font-size:85%;font:85%"|Low Gothic, High Gothic,\nLingua-technis\n|-\n|cellpadding=2 cellspacing=1 style="font-size:85%;font:85%"|Major Species:\n|cellpadding=2 cellspacing=1 style="font-size:85%;font:85%"|Human, Abhuman species\n|-\n|cellpadding=2 cellspacing=1 style="font-size:85%;font:85%"|Type of Government' metadata={'title': 'Imperium of Man'}
page_conten

In [ ]:
len(texts[0].page_content)

918

## Populate vectorstore

First initialise the vector store

In [ ]:
from langchain.vectorstores import Chroma
from langchain.schema.document import Document
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-mpnet-base-v2") #embedding_functions.DefaultEmbeddingFunction()

persist_directory = 'db'
vectorstore = Chroma("langchain_store", embedding, persist_directory=persist_directory)

This adds the documents to the vector store



In [ ]:
vectorstore.add_documents(documents=[Document(**document) for document in documents]);

In [ ]:
# vectorstore.similarity_search("emperor", 5)

# Model

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatAnthropic

In [ ]:
llm = ChatAnthropic(anthropic_api_key=os.getenv("ANTHROPIC_API_KEY"))
#llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.)

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k":4})
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff",retriever=retriever)

In [ ]:
question = "From the provided context, can you write a paragraph describing the greatest acheivements of the emperor"

answer = qa.run(question)
print(answer)

 Here is a paragraph describing some of the Emperor's greatest achievements during the Great Crusade:

The Emperor achieved many momentous feats during the approximately 200 years of the Great Crusade. After unifying Terra, he conquered Mars and the rest of the Solar System. He created the Primarchs and Space Marine Legions who served as the vanguard of his forces. World after world fell to the Emperor's armies as he gradually reunited humanity across the galaxy. The Emperor himself commanded some of the greatest victories, including the Battle of Gyros-Thravian where he fought alongside Horus, Rogal Dorn and Mortarion against the powerful Ork warboss Ghazghkull. The pinnacle of his success was marked by Horus' great victory against the Orks at Ullanor, after which the Emperor named him Warmaster. Through his leadership, strategies and sheer force of will, the Emperor brought mankind to its apex during the Great Crusade.


In [ ]:
question = "From the provided context, can you write a paragraph the governance of the imperium and it's greatest threats going into the 41st millenium"

answer = qa.run(question)
print(answer)

 Based on the provided context, here is a paragraph about the governance of the Imperium and its greatest threats going into the 41st millennium:

The Imperium is governed by the High Lords of Terra, who rule on behalf of the Emperor from the Imperial Palace on Terra. However, in late M41, Roboute Guilliman returned and instituted reforms, creating the Logos Historica Verita to record an accurate history of the Imperium. This brought him into conflict with traditionalists who want to restore the pre-Guilliman Imperium Eterna. The greatest threats facing the Imperium are the forces of Chaos and xenos like the Tyranids, Orks, and Necrons, as well as internal dissent between reformists like Guilliman and traditional High Lords. As the Imperium entered the 41st millennium, it remained riven by the Great Rift and was still recovering from the ravages of the 13th Black Crusade.


# OLD: using default chroma

this is from my earlier experimenting using chroma directly, but it's nice using it through langchain, so I've stuck with that (see above)

In [ ]:
# # Import Chroma and instantiate a client. The default Chroma client is ephemeral, meaning it will not save to disk.
# import chromadb

# client = chromadb.Client()

In [ ]:
# # Create a new Chroma collection to store the supporting evidence. We don't need to specify an embedding fuction, and the default will be used.
# collection = client.create_collection("sciq_supports")

In [ ]:
# # Embed and store the first 100 supports for this demo
# collection.add(
#     ids=[str(i) for i in range(0, 100)],  # IDs are just strings
#     documents=dataset["support"][:100],
#     metadatas=[{"type": "support"} for _ in range(0, 100)
#     ],
# )

In [ ]:
# dataset["support"][0]

In [ ]:
# results = collection.query(
#     query_texts=dataset["question"][:10],
#     n_results=1)

In [ ]:
# dataset["question"][0]

In [ ]:
# collection.query(
#     query_texts="Radiation",
#     n_results=1)

In [ ]:
# # Print the question and the corresponding support
# for i, q in enumerate(dataset['question'][:10]):
#     print(f"Question: {q}")
#     print(f"Retrieved support: {results['documents'][i][0]}")
#     print()